In [ ]:
import os, sys
sys.path.append(os.path.abspath(".."))

import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

# Load the trained model
model = tf.keras.models.load_model("../models/plant_disease.keras")

# Load one image for inference
img_path = "../data/processed/val/Tomato___Late_blight/0acdc2b2-0dde-4073-8542-6fca275ab974___RS_LB 4857.JPG"
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)

pred = model.predict(img_array)
pred_idx = np.argmax(pred[0])
print("Predicted class index:", pred_idx)